In [1]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

from scipy.integrate import odeint

import matplotlib.pyplot as plt
import numpy as np

import math

from matplotlib import figure
from ipywidgets import interact

import ternary

In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [180]:
# Parameters widgets

tF = widgets.FloatSlider(min=0., max=1., step=0.1, value=1.)
tD = widgets.FloatSlider(min=0., max=1., step=0.1, value=0.3)
tS = widgets.FloatSlider(min=0., max=1., step=0.1, value=0.3)
muftd = widgets.FloatSlider(min=0., max=0.01, step=0.001, value=0.001,readout_format='.3f')
mudtf = widgets.FloatSlider(min=0., max=0.1, step=0.01, value=0.01)
mufts = widgets.FloatSlider(min=0., max=0.01, step=0.0001, value=0.0001,readout_format='.4f')
mustf = widgets.FloatSlider(min=0., max=0.01, step=0.0001, value=0.0001,readout_format='.4f')
Kwid = widgets.FloatLogSlider(value=1e12,base=10,min=2,max=15,step=1)
nbrcycles = widgets.IntSlider(min=1., max=100., step=1., value=10.)
lencycle = widgets.FloatSlider(min=1., max=100., step=0.1, value=4.)
n0bactos = widgets.FloatLogSlider(value=1e5,base=10,min=2,max=15,step=1)
Finit = widgets.FloatSlider(min=0., max=1., step=0.01, value=1.)
Dinit = widgets.FloatSlider(min=0., max=1., step=0.01, value=0.)
bottlesize = widgets.FloatSlider(min=0., max=1., step=0.01, value=0.01)


In [181]:
text_m1 = widgets.HTML(value="<h2>Long-term evolutionary fate of  spontaneously duplicated tRNA genes in bacteria</h2>")
text_0 = widgets.HTML(value="<h3>Founder strain replication time</h3>")
text_1 = widgets.HTML(value="<h3>Duplication strain replication time</h3>")
text_2 = widgets.HTML(value="<h3>SNP strain replication time</h3>")
text_3 = widgets.HTML(value="<h3>Mutation rate F to D</h3>")
text_4 = widgets.HTML(value="<h3>Mutation rate D to F</h3>")
text_5 = widgets.HTML(value="<h3>Mutation rate F to S</h3>")
text_6 = widgets.HTML(value="<h3>Mutation rate S to F</h3>")
text_7 = widgets.HTML(value="<h3>Carrying capacity</h3>")
text_8 = widgets.HTML(value="<h3>Number of passages</h3>")
text_9 = widgets.HTML(value="<h3>Time between two passages</h3>")
text_10 = widgets.HTML(value="<h3>Initial (total) number of bacteria</h3>")
text_11 = widgets.HTML(value="<h3>Initial proportion of founder (del) strain</h3>")
text_12 = widgets.HTML(value="<h3>Initial proportion of duplication strain</h3>")
text_13 = widgets.HTML(value="<h3>Bottleneck size (dilution factor)</h3>")

In [182]:
vbox_text = widgets.VBox([text_0,tF, text_1, tD, text_2, tS, text_3, muftd,text_4, mudtf, text_5, mufts, text_6, mustf, text_7, Kwid,text_8,nbrcycles,text_9,lencycle,text_10,n0bactos,text_11,Finit,text_12,Dinit,text_13,bottlesize])

In [183]:
def solving_function(tt, params, func, y0):
    n = len(y0)-1
    
    K,rF,rD,rS,mFD,mFS,mDF,mSF = params

    sol = odeint(func, y0, tt, args=(K,rF,rD,rS,mFD,mFS,mDF,mSF,), hmax=0.001)

    
    return tt, sol

In [184]:
def grad(y,t,K,rF,rD,rS,mFD,mFS,mDF,mSF):
    
    '''Gives the derivative with respect to time of the different subpopulations: R, Du, S'''
    
    dFdt = (1-(y[0]+y[1]+y[2])/K)*(rF*(1-mFD-mFS)*y[0]+rD*mDF*y[1]+rS*mSF*y[2])
    
    dDdt = (1-(y[0]+y[1]+y[2])/K)*(rD*(1-mDF)*y[1]+rF*mFD*y[0])
    
    dSdt = (1-(y[0]+y[1]+y[2])/K)*(rS*(1-mSF)*y[2]+rF*mFS*y[0])
    
    dydt = np.array([dFdt,dDdt,dSdt])
    
    return dydt

In [252]:
# button visualize 1

button_send = widgets.Button(
                description='Visualize',
                tooltip='Visualize',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        
        rF = np.log(2)/tF.value
        rD = np.log(2)/tD.value
        rS = np.log(2)/tS.value
        
        mFD = muftd.value/np.log(2)
        mDF = mudtf.value/np.log(2)
        mFS = mufts.value/np.log(2)
        mSF = mustf.value/np.log(2)
        
        K = Kwid.value
        
        tcycle = lencycle.value
        bottleneck = bottlesize.value
        ncycle = nbrcycles.value
        npoint=1000

        n0 = n0bactos.value
        y0 = np.array([n0*Finit.value,n0*Dinit.value,n0*(1-Finit.value-Dinit.value)])

        
        t0temp = 0
        y0temp = y0
        tftemp = tcycle
        tttemp = np.linspace(t0temp,tftemp,npoint)
        
        params = [K,rF,rD,rS,mFD,mFS,mDF,mSF]
        
        Fsol=[]
        Dsol=[]
        Ssol=[]
        
        for i in range(ncycle):
            soltemp = solving_function(tttemp,params,grad,y0temp)
            solformat = np.transpose(soltemp[1])
            Fsol = np.append(Fsol,solformat[0][0:-1])
            Dsol = np.append(Dsol,solformat[1][0:-1])
            Ssol = np.append(Ssol,solformat[2][0:-1])
            y0temp = np.array([Fsol[-1]*bottleneck,Dsol[-1]*bottleneck,Ssol[-1]*bottleneck])

            
        ttplot = np.linspace(t0temp,tcycle*ncycle,ncycle*(npoint-1))
    
        # Proportions
        plt.plot(ttplot, Fsol/(Fsol+Dsol+Ssol),label='Founder strain')
        plt.plot(ttplot, Dsol/(Fsol+Dsol+Ssol),label='Duplication strain')
        plt.plot(ttplot, Ssol/(Fsol+Dsol+Ssol),label='SNP strain')
        
        plt.legend()
        plt.xlabel('time (days)')
        plt.ylabel('proportions')
        plt.tight_layout()     
        A = plt.figure()
        
        # Absolute numbers
        plt.plot(ttplot, Fsol,label='Founder strain')
        plt.plot(ttplot, Dsol,label='Duplication strain')
        plt.plot(ttplot, Ssol,label='SNP strain')
        
        plt.legend()
        plt.xlabel('time (days)')
        plt.ylabel('abundance')
        plt.semilogy()
        
        #plt.tight_layout()
        
        #B = plt.figure()  
        
        # Simplex
        founder=Fsol/(Fsol+Dsol+Ssol)
        dup=Dsol/(Fsol+Dsol+Ssol)
        snp=Ssol/(Fsol+Dsol+Ssol)
        
        scale = 1.0
        figure, tax = ternary.figure(scale=scale)
    
        tax.boundary(linewidth=2.0)
        tax.gridlines(color="blue", multiple=0.2)
        tax.set_title("Plotting of sample trajectory data", fontsize=20)
        figure.set_size_inches(10, 8)
        points=[]
        for i in range(len(tttemp)):
            points.append((founder[i],dup[i],snp[i]))
        tax.plot(points, linewidth=2.0, label="Curve")
        fontsize = 16
        offset = 0.3
        tax.set_title("\n", fontsize=fontsize)
        tax.right_corner_label("Founder", fontsize=fontsize,offset=0.3)
        tax.top_corner_label("Dup", fontsize=fontsize,offset=0.25)
        tax.left_corner_label("SNP", fontsize=fontsize,offset=0.2)
    
        tax.ticks(axis='lbr', multiple=.2, linewidth=1, offset=0.03,tick_formats="%.1f",fontsize=fontsize)
        tax.get_axes().axis('off')
        tax.clear_matplotlib_ticks()
        
        C = tax.show()
        
        return (C)
    

button_send.on_click(on_button_clicked)

vbox_result = widgets.HBox([button_send, output])

In [253]:
page = widgets.VBox([text_m1,widgets.HBox([vbox_text,vbox_result])])
display(page)

In [254]:
!pip freeze > requirements.txt